In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Importing all the necessary modules for Deepfake detection
from builtins import range, input
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

In [17]:
# Loading Data
image_data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/CT')

Found 8054 files belonging to 2 classes.


In [ ]:
# Initializing a numpy iterator for the image_data dataset
data_iterator = image_data.as_numpy_iterator()
# Get the next batch of images  from the iterator
batch = data_iterator.next()
# Check the batch size
batch_size = batch.shape[0]
print(f"Batch size: {batch_size}")

In [ ]:
# looking at  images from dataset
fig, ax = plt.subplots(ncols=4,nrows=4, figsize=(24,24))
for idx, img in enumerate(batch[0][:7]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# Splitting data for training, validation and testing
import random
img_training = int(0.7 * len(image_data))
img_validation = int(0.15 * len(image_data))
img_testing = int(0.15 * len(image_data))

# Randomly shuffling the images
random.shuffle(image_data)

# Split the dataset into training, validation, and testing sets
train = image_data[:img_training]
val = image_data[img_training:img_training+img_validation]
test = image_data[img_training+img_validation:]

In [9]:
# Building CNN Model
model =Sequential()

In [13]:
# Convolutional Layer1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))

# Max Pooling Layer1
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer2
model.add(Conv2D(64, (3, 3), activation='relu'))

# Max Pooling Layer2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer3
model.add(Conv2D(128, (3, 3), activation='relu'))

# Max Pooling Layer3
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer4
model.add(Conv2D(256, (5, 5), activation='relu'))

# Max Pooling Layer4
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten Layer
model.add(Flatten())

# Dense Layer1
model.add(Dense(512, activation='relu'))

# Dropout Layer
model.add(Dropout(0.5))
# Dense Layer2
model.add(Dense(256, activation='relu'))

# Dropout layer
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(1, activation='sigmoid'))


In [14]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0